In [1]:
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY,
)

In [4]:
# Calculator tool
def calculator(expression: str):
    """Evaluate a maths expression"""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

In [8]:
from langgraph.prebuilt import ToolNode

model_with_tools = llm.bind_tools([calculator])
tool_node = ToolNode([calculator])

In [12]:
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [13]:
from langgraph.graph import END

# Nodes
def call_model(state: State) -> State:
    messages = state['messages']
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: State):
    """Decide if finish or use a tool"""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls: # if model requests a tool call
        return "tools"
    return END

In [14]:
from langgraph.graph import StateGraph
from langgraph.graph import START, END

# Build the graph
workflow = StateGraph(State)

workflow.add_node("call_model", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "call_model")
workflow.add_conditional_edges("call_model", should_continue, ["tools", END])
workflow.add_edge("tools", "call_model")

In [15]:
# Compile graph
graph = workflow.compile()

In [16]:
# TEST
query = {"role": "user", "content": "What is 36 * 33?"}
result = graph.invoke({"messages": query})

print("\nCHAT:")
for msg in result["messages"]:
    print(msg)


CHAT:
content='What is 36 * 33?' additional_kwargs={} response_metadata={} id='364f0e4d-f34f-416a-b1fe-02d0915e3ab7'
content='' additional_kwargs={'function_call': {'name': 'calculator', 'arguments': '{"expression": "36 * 33"}'}, '__gemini_function_call_thought_signatures__': {'7788eb59-408d-4d07-adf8-d182b346f53f': 'CuIBAXLI2nxHK3cm5XFGzVNw0JdckeC2YNlRfOA+b2eBwGInxU26Rk+6HTEYiT61EHD8F22BJyMTLrjoolMxUX0aYpECgi+UCxnMZ8AWFZGuCTnjIqh+o78mOmHMsHO8p8gUgvZY7COXfaOkYhs1HsGoO6A6Jnh7RCFtQQeWOG8DP+xS+NdKa5rtae+B3kT5b0TrdEQvWoo/Dvqwd972CfgaLDEos9LGTxX+8ZOSPkOhHHJfcAyjyNUSiIgBrwDgT5NgzpTieq6HRwKIzxLvKrXGkbVNNT+PPkdU1pE69rl2zNTysA=='}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b1927-7911-7971-820b-53a8d8846515-0' tool_calls=[{'name': 'calculator', 'args': {'expression': '36 * 33'}, 'id': '7788eb59-408d-4d07-adf8-d182b346f53f', 'type': 'tool_call'}] usage_metadata={'input_tokens': 45, 'output_